In [ ]:
import datetime, json, os, re, requests, subprocess

def exec_ipynb(filename_or_url):
    nb = (requests.get(filename_or_url).json() if re.match(r'https?:', filename_or_url) else json.load(open(filename_or_url)))
    if(nb['nbformat'] >= 4):
        src = [''.join(cell['source']) for cell in nb['cells'] if cell['cell_type'] == 'code']
    else:
        src = [''.join(cell['input']) for cell in nb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']

    tmpname = '/tmp/%s-%s-%d.py' % (os.path.basename(filename_or_url),
                                    datetime.datetime.now().strftime('%Y%m%d%H%M%S%f'),
                                    os.getpid())
    src = '\n\n\n'.join(src)
    open(tmpname, 'w').write(src)
    code = compile(src, tmpname, 'exec')
    exec(code, globals())

exec_ipynb('../utils.ipynb')


In [ ]:
# Test that results are returned in order of submission

def sleep_and_return(i):
    # Complete futures out of order by each future sleepi
    time.sleep(0.1 * (i * 7 % 10))
    return i

def test_pool_type(pool_type):
    pool = pool_type(10)
    for i in range(10):
        pool.submit(sleep_and_return, i)

    results = pool.shutdown()
    assert results == list(range(10))

test_pool_type(SimpleThreadPoolExecutor)
test_pool_type(SimpleProcessPoolExecutor)
